In [1]:
%cd COVID19-classifier/

/home/jupyter/COVID19-classifier


In [2]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import os
import torch
import pandas as pd
import numpy as np
from torch import optim, nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
from utils import *
from models import *

In [3]:
import pixiedust

Pixiedust database opened successfully


In [4]:
DN = DenseNet121().cuda()
VGG = VGG16().cuda()

In [5]:
criterion = nn.BCELoss()
optimizer_DN = optim.Adam(DN.parameters(), lr=0.001)
optimizer_VGG = optim.Adam(VGG.parameters(), lr=0.001)

In [6]:
transform1 = transforms.Resize((224, 224))
transform2 = transforms.Compose([transforms.Resize((224, 224)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(10),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                      std=[0.5, 0.5, 0.5])])
data = datasets.ImageFolder(root='dataset/train', transform=transform1)
raw_data = np.array([np.array(image[0], dtype="float") for image in data])
raw_minority_data = np.array([np.array(image[0], dtype="float") for image in data if image[1]==0])

In [7]:
transform_smote = CustomCompose([transforms.Resize((224, 224)),
                                 Smote(raw_minority_data, 5),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(10),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                      std=[0.5, 0.5, 0.5])])
transform_SP_minority = CustomCompose([transforms.Resize((224, 224)),
                                       SampleParing(raw_data, minority_only=True),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomRotation(10),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                        std=[0.5, 0.5, 0.5])])
transform_SP = CustomCompose([transforms.Resize((224, 224)),
                              SampleParing(raw_data, minority_only=False),
                              transforms.RandomHorizontalFlip(),
                              transforms.RandomRotation(10),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                   std=[0.5, 0.5, 0.5])])
transform_RICAP = CustomCompose([transforms.Resize((224, 224)),
                                 RICAP(data, beta=1),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(10),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                      std=[0.5, 0.5, 0.5])])
transform_Maj = CustomCompose([transforms.Resize((224, 224)),
                               Majority(raw_data),
                               transforms.RandomHorizontalFlip(),
                               transforms.RandomRotation(10),
                               transforms.ToTensor(),
                               transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                    std=[0.5, 0.5, 0.5])])
data_smote = CustomFolder(root='dataset/train', transform=transform_smote)
data_SP_minority = CustomFolder(root='dataset/train', transform=transform_SP_minority)
data_SP = CustomFolder(root='dataset/train', transform=transform_SP)
data_RICAP = CustomFolder(root='dataset/train', transform=transform_RICAP)
data_Maj = CustomFolder(root='dataset/train', transform=transform_Maj)
data_orginal = CustomFolder(root='dataset/train', transform=transform2)

In [ ]:
dataloader = DataLoader(data_smote, batch_size=16, shuffle=True, num_workers=4)
for epoch in range(100):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.type(torch.FloatTensor).cuda()
        optimizer_DN.zero_grad()
        outputs = DN(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_DN.step()

        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 20))
            print(outputs)
            running_loss = 0.0
print('Finished Training')

[1,    20] loss: 0.317
tensor([[0.0115],
        [0.9795],
        [0.9773],
        [0.9456],
        [0.0048],
        [0.9028],
        [0.8504],
        [0.9294],
        [0.4994],
        [0.1564],
        [0.9381],
        [0.9765],
        [0.7148],
        [0.9892],
        [0.4736],
        [0.7283]], device='cuda:0', grad_fn=<SigmoidBackward>)


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


[2,    20] loss: 0.303
tensor([[0.9232],
        [0.1370],
        [0.9829],
        [0.9982],
        [0.9119],
        [0.9105],
        [0.8851],
        [0.7606],
        [0.7497],
        [0.9441],
        [0.1037],
        [0.2425],
        [0.8365],
        [0.9961],
        [0.9877],
        [0.8786]], device='cuda:0', grad_fn=<SigmoidBackward>)
[3,    20] loss: 0.274
tensor([[0.9831],
        [0.2357],
        [0.8608],
        [0.0917],
        [0.8674],
        [0.9967],
        [0.8477],
        [0.9692],
        [0.9052],
        [0.9288],
        [0.9627],
        [0.1089],
        [0.8762],
        [0.9743],
        [0.9158],
        [0.0739]], device='cuda:0', grad_fn=<SigmoidBackward>)
[4,    20] loss: 0.253
tensor([[0.5556],
        [0.0343],
        [0.9893],
        [0.9857],
        [0.9819],
        [0.7517],
        [0.2314],
        [0.2400],
        [0.9917],
        [0.8467],
        [0.9856],
        [0.0820],
        [0.9997],
        [0.2024],
        [0.99